<a href="https://colab.research.google.com/github/HarshShinde0/url-to-gdrive/blob/main/url_to_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests pydrive tqdm -q

# **Download File from Link**

In [5]:
import requests
from tqdm import tqdm  # For a clean progress bar

# Step 1: Get URL from user
url = input("Enter the download URL: ").strip()
filename = url.split('/')[-1].split('?')[0]

# Step 2: Download with progress bar
def download_file(url, filename):
    print(f"Downloading from {url}...")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        block_size = 8192  # 8 KB
        t = tqdm(total=total_size, unit='B', unit_scale=True, desc=filename)
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=block_size):
                f.write(chunk)
                t.update(len(chunk))
        t.close()
    print(f"✅ Download complete: {filename}")
    return filename

# Run download
download_file(url, filename)

Enter the download URL: https://ftp.osuosl.org/pub/osgeo/download/qgis/windows/QGISQT6-OSGeo4W-3.42.3-1.msi?MU


QGISQT6-OSGeo4W-3.42.3-1.msi: 100%|██████████| 1.31G/1.31G [00:21<00:00, 60.3MB/s]

✅ Download complete: QGISQT6-OSGeo4W-3.42.3-1.msi


'QGISQT6-OSGeo4W-3.42.3-1.msi'

# **Upload to Google Drive**

In [8]:
# Authenticate
from google.colab import auth
auth.authenticate_user()

# Imports
import os
import google.auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Get user credentials and build the Drive API service
creds, _ = google.auth.default()
service = build('drive', 'v3', credentials=creds)

# Ask for the file to upload
filename = input("Enter the filename to upload to Google Drive: ").strip()

# Upload file to Google Drive
if os.path.exists(filename):
    print(f"Uploading {filename}...")
    file_metadata = {'name': filename}
    media = MediaFileUpload(filename, resumable=True)
    uploaded_file = service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()

    # Make the file publicly accessible
    service.permissions().create(
        fileId=uploaded_file['id'],
        body={'type': 'anyone', 'role': 'reader'}
    ).execute()

    # Display shareable link
    print("✅ Upload complete!")
    print(f"🔗 File link: https://drive.google.com/file/d/{uploaded_file['id']}/view")
else:
    print("❌ File does not exist.")

Enter the filename to upload to Google Drive: /content/QGISQT6-OSGeo4W-3.42.3-1.msi
Uploading /content/QGISQT6-OSGeo4W-3.42.3-1.msi...
✅ Upload complete!
🔗 File link: https://drive.google.com/file/d/1tHcqbmqoywiq0cWJViOZLRTdPiK4GKrW/view
